# 拆分excel表格，然后进行邮件发送
## 一、拆分excel

In [31]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import xlwings as xw
file_path = r'发托管-顺德农商分配计算表202103.xlsx'
sheet_name = '收入分配明细表2021-3-15' # 需要拆分的sheet名称
app = xw.App(visible = True, add_book = False) # 启动Excel程序
workbook = app.books.open(file_path) # 打开来源工作簿
worksheet = workbook.sheets[sheet_name] # 选中要拆分的数据所在sheet

value = worksheet.range('A41').expand('table').value # 读取需要拆分的所有数据，A41单元格是数据的起始单元格，根据实际情况修改
data = dict() # 创建一个空字典，用于按要求分类储存数据

for i in range(len(value)): # 按行遍历工作表数据
    client_name = value[i][9] # 获取分类依据的数据，两个[]的数字代表行和列，从0开始
    if client_name not in data: #判断字典中是否已有当前客户名称
        data[client_name] = [] # 若没有，就创建一个当前客户名称的空列表，用来储存当前行数据
    data[client_name].append(value[i]) # 将当前行数据追加到当前行客户名称对应的列表中

for key,value in data.items(): # 按客户名称遍历分类后的数据
    new_workbook = xw.books.add() # 新建工作簿
    new_worksheet = new_workbook.sheets.add(key) # 在新工作簿中新建工作表，并且命名为客户名称
    new_worksheet['A1'].value = worksheet['A40:M40'].value # 将要拆分的列标题复制到第一行,根据实际情况修改
    new_worksheet['A2'].value = value # 将当前客户名称下的数据复制到新建的工作表
    new_worksheet.autofit() #自动调整格式
    new_workbook.save('{}.xlsx'.format(key)) # 保存为新工作簿，命名为客户名称，这里也可以加上当前日期或其他内容，根据实际需要调整
app.quit() # 退出Excel程序


## 二、邮件发送

In [10]:
 #---------以下是进行邮件操作，用到的模块为win32----------#
import win32com.client as win32

outlook = win32.Dispatch('Outlook.Application')
Mail_Item = outlook.CreateItem(0)
Mail_Item.Recipients.Add('zhaoyunchao@utrust.cn')
# 应先在邮箱通讯录中建立邮件组，直接输入邮件组名字可以发送
Mail_Item.Subject = 'test'
Mail_Item.BodyFormat = 2
Mail_Item.HtmlBody = '''
                     Hello Guys,<br/>
                     &nbsp;&nbsp;&nbsp;&nbsp;This a Mail send by Jerry'''
Mail_Item.Attachments.Add(r'D:\learnpython\excel_examples\佛山市三水凤铝铝业有限公司.xlsx')
Mail_Item.Save() # 保存草稿
Mail_Item.Send()
#------------邮件发送完成----------------------------------------------------------#

将上述过程定义成函数，方便复用

In [ ]:
def sendmail():
    sub = 'outlook python mail test' #邮件标题，根据实际情况修改
    body = 'my test\r\n my python mail' #邮件正文，根据实际情况修改
    outlook = win32.Dispatch('outlook.application') 打开outlook
    receivers = ['xxx'] #收件人列表，可以设置为从指定的excel表中自动读取
    mail = outlook.CreateItem(0) #新建邮件
    mail.To = receivers[0] #发送给列表中的第一个，以后可以使用for循环，实现分别发送
    mail.CC = receivers[1] # 抄送人
    mail.Subject = sub.decode('utf-8') # 邮件主题，设置了编码格式，解决乱码问题
    mail.Body = body.decode('utf-8') #邮件正文，，设置了编码格式，解决乱码问题
    mail.Attachments.Add('C:\Users\xxx\Desktop\git_auto_pull_new.py') #附件路径，可以通过列表形式来读取，跟收件人匹配
    mail.Send()


下面代码我觉得网友写的不错，抄下来备用，来源：https://blog.csdn.net/songlh1234/article/details/84031297

In [ ]:
import win32com.client as win32
import datetime, os
 
addressee = 'test01@qq.com'+';'+'test02@jd.com'#收件人邮箱列表
cc = 'test02@163.com'+';'+'test03@alibaba.com'#抄送人邮件列表
mail_path = os.path.join(r'C:\Users\songlihui\PycharmProjects\test001keshanchu', 'result', 'report.html')#获取测试报告路径
 
class send_email():
    def outlook(self):
        olook = win32.Dispatch("outlook.Application")#固定写法
        mail = olook.CreateItem(win32.constants.olMailItem)#固定写法
        mail.To = addressee#收件人
        mail.CC = cc#抄送人
        # mail.Recipients.Add(addressee)
        mail.Subject = str(datetime.datetime.now())[0:19]+'XXX反馈报告'#邮件主题
        mail.Attachments.Add(mail_path, 1, 1, "myFile")
        read = open(mail_path, encoding='utf-8')#打开需要发送的测试报告附件文件
        content = read.read()#读取测试报告文件中的内容
        read.close()
        mail.Body = content#将从报告中读取的内容，作为邮件正文中的内容
        mail.Send()#发送
 
 
if __name__ == '__main__':
    send_email().outlook()
    print("send email ok!!!!!!!!!!")

## 三、看了其他网友的代码后，逐渐有了实现的路径，下面开始，逐步来实现我的实际情况

In [1]:
import xlwings as xw
import win32com.client as win32
import  os
# 完成时间 2021/6/12 端午假期
# 首先定义所有需要用到的文件路径
file_path = '台账模板.xlsx' # 分配表文件所在路径，由于我将文件都拷贝到当前文件夹，所以直接写文件名即可
sheet_name = '模板' # 需要拆分的sheet名称，根据实际情况确定
file_path_investors = r'D:\learnpython\excel_examples\投资者信息统计.xlsx' # 投资者信息所在路径
report = r'D:\learnpython\excel_examples\XX项目受托报告202106.pdf'


#打开excel，读取分配总表数据
app = xw.App(visible = False, add_book = False) # 启动Excel程序
workbook = app.books.open(file_path) # 打开分配表来源工作簿
worksheet = workbook.sheets[sheet_name] # 选中要拆分的数据所在sheet
value = worksheet.range('B38').expand('table').value # 读取需要拆分的所有数据，B38单元格是数据的起始单元格，根据实际情况修改
data = dict() # 创建一个空字典，用于按要求分类储存数据
#打开投资者信息数据
workbook_1 = app.books.open(file_path_investors) # 打开投资者信息工作簿
worksheet_1 = workbook_1.sheets['Sheet1'] # 选中提取的数据所在sheet
value_1 = worksheet_1.range('A1').expand('table').value # 读取需要投资者信息数据
# 拆分分配数据
for i in range(len(value)): # 按行遍历工作表数据
    client_name = value[i][0] # 获取分类依据的数据，两个[]的数字代表行和列，从0开始
    if client_name not in data: #判断字典中是否已有当前客户名称
        data[client_name] = [] # 若没有，就创建一个当前客户名称的空列表，用来储存当前行数据
    data[client_name].append(value[i]) # 将当前行数据追加到当前行客户名称对应的列表中

for key,value in data.items(): # 按客户名称遍历分类后的数据
    new_workbook = xw.books.add() # 新建工作簿
    new_worksheet = new_workbook.sheets.add(key) # 在新工作簿中新建工作表，并且命名为客户名称
    new_worksheet['A1'].value = worksheet['B37:H37'].value # 将要拆分的列标题复制到第一行,根据实际情况修改
    new_worksheet['A2'].value = value # 将当前客户名称下的数据复制到新建的工作表
    new_worksheet.autofit() #自动调整格式    
    # 保存为新工作簿，命名为客户名称，这里也可以加上当前日期或其他内容，根据实际需要调整    
    new_workbook.save(r'D:\learnpython\excel_examples\分配表\{}.xlsx'.format(key)) 
    print('{}已拆分完毕'.format(key))

    # 匹配投资者收件箱，发送邮件
    for j in range(len(value_1)): # 对投资者信息遍历
        if value_1[j][0] in key: # 投资者名称如果和分配表文件名一致
            # 发送邮件
            outlook = win32.Dispatch('Outlook.Application') #初始化outlook
            mail = outlook.CreateItem(0) # 固定写法
            
            mail.To = value_1[j][4] # 邮箱收件人名称就是邮箱
            mail.CC = ''
            mail.Subject = 'XX项目分配邮件通知'
            mail.HtmlBody = """
            <p><b>尊敬的投资者：</b></p>
            <p>您好，请查收贵司本次投资分配明细表，以及受托报告，详见附件。</p>
            <p>如有任何问题，请及时联系我们。</p>
            <font color="gray" size="2">赵运超</font><br>
            <font color="gray" size="2">手机：18924588577/13902319827</font><br/>
            """
            path1 = r'D:\learnpython\excel_examples\分配表\{}.xlsx'.format(key)
            path2 = report
             
            mail.Attachments.Add(path1) # 附件1，分配表
            mail.Attachments.Add(path2) # 附件2，受托报告

        #     mail.Send()#发送# 应先在邮箱通讯录中建立邮件组，直接输入邮件组名字可以发送
            mail.Save() #保存草稿
            print('{}邮件已发送'.format(key))
#             print('找不到投资者{}，请确保分配表中的投资者名称与信息表中完全一致'.format(key))
    
app.quit() # 退出Excel程序

print("*"*27)
print('未提示已发送的，请确保分配表中的投资者名称与信息表中完全一致')
print("已全部拆分完毕,并完成邮件发送")
print("*"*27)

北京银行已拆分完毕
北京银行邮件已发送
兴业银行已拆分完毕
中信银行已拆分完毕
上饶银行已拆分完毕
长沙银行已拆分完毕
广州银行已拆分完毕
台州银行已拆分完毕
小计已拆分完毕
招商证券已拆分完毕
兴业银行长春分行已拆分完毕
东莞银行已拆分完毕
国泰君安证券已拆分完毕
国盛证券已拆分完毕
越秀金控资本管理公司已拆分完毕
中金公司已拆分完毕
兴业资产管理公司已拆分完毕
***************************
未提示已发送的，请确保分配表中的投资者名称与信息表中完全一致
已全部拆分完毕,并完成邮件发送
***************************


# 四、python不调用客户端直接发邮件
## 示例1代码
https://zhuanlan.zhihu.com/p/24180606

In [5]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage

#设置登录及服务器信息
mail_host = 'smtp.utrust.cn'
mail_user = 'zhaoyunchao@utrust.cn'
mail_pass = 'Yc888888'
sender = 'zhaoyunchao@utrust.cn'
receivers = ['15757065@qq.com']

#设置eamil信息
#添加一个MIMEmultipart类，处理正文及附件
message = MIMEMultipart()
message['From'] = sender
message['To'] = receivers[0]
message['Subject'] = 'title'
#推荐使用html格式的正文内容，这样比较灵活，可以附加图片地址，调整格式等
with open('abc.html','rb') as f:
    content = f.read()
#设置html格式参数
part1 = MIMEText(content,'html','utf-8')
#添加一个txt文本附件
with open('abc.txt','rb')as h:
    content2 = h.read()
#设置txt参数
part2 = MIMEText(content2,'plain','utf-8')
#附件设置内容类型，方便起见，设置为二进制流
part2['Content-Type'] = 'application/octet-stream'
#设置附件头，添加文件名
part2['Content-Disposition'] = 'attachment;filename="abc.txt"'
#添加照片附件
# with open('1.png','rb')as fp:
#     picture = MIMEImage(fp.read())
#     #与txt文件设置相似
#     picture['Content-Type'] = 'application/octet-stream'
#     picture['Content-Disposition'] = 'attachment;filename="1.png"'
#将内容附加到邮件主体中
message.attach(part1)
message.attach(part2)
# message.attach(picture)

#登录并发送
try:
    smtpObj = smtplib.SMTP()
    smtpObj.connect(mail_host,25)
    smtpObj.login(mail_user,mail_pass)
    smtpObj.sendmail(
        sender,receivers,message.as_string())
    print('success')
    smtpObj.quit()
except smtplib.SMTPException as e:
    print('error',e)

success


## 示例2代码
https://blog.csdn.net/yuting209/article/details/105424833

In [ ]:
"""__author__=余婷"""
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.header import Header

# 1. 连接邮箱服务器
con = smtplib.SMTP_SSL('smtp.163.com', 465)

# 2. 登录邮箱
con.login('XXXX@163.com', 'XXXXX')

# 2. 准备数据
# 创建邮件对象
msg = MIMEMultipart()

# 设置邮件主题
subject = Header('找回密码', 'utf-8').encode()
msg['Subject'] = subject

# 设置邮件发送者
msg['From'] = 'XXXX@163.com <XXXX@163.com>'

# 设置邮件接受者
msg['To'] = '726550822@qq.com'

# 添加文字内容
text = MIMEText('忘记密码需要找回密码', 'plain', 'utf-8')
msg.attach(text)

# 3.发送邮件
con.sendmail('xxxx@163.com', '726550822@qq.com', msg.as_string())
con.quit()


## 发送html格式文本

In [ ]:
"""__author__=余婷"""
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.header import Header

# 1. 连接邮箱服务器
con = smtplib.SMTP_SSL('smtp.163.com', 465)

# 2. 登录邮箱
con.login('xxxx@163.com', 'xxxxxx')

# 2. 准备数据
# 创建邮件对象
msg = MIMEMultipart()

# 设置邮件主题
subject = Header('找回密码', 'utf-8').encode()
msg['Subject'] = subject

# 设置邮件发送者
msg['From'] = 'xxxx@163.com <xxxx@163.com>'

# 设置邮件接受者
msg['To'] = '726550822@qq.com'

# 添加html内容
content = """
<h2>我是正文中的标题</h2>
<p>邮件正文描述性文字1</p>
<p>邮件正文描述性文字2</p>
<img src='https://www.baidu.com/img/bd_logo1.png'>
<center>百度图片</center>
<a href='https://www.baidu.com'>百度一下</a>
"""
html = MIMEText(content, 'html', 'utf-8')
msg.attach(html)

# 3.发送邮件
con.sendmail('xxxx@163.com', '726550822@qq.com', msg.as_string())
con.quit()


## 发送图片内容的代码

In [ ]:
"""__author__=余婷"""
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.header import Header

# 1. 连接邮箱服务器
con = smtplib.SMTP_SSL('smtp.163.com', 465)

# 2. 登录邮箱
con.login('xxxx@163.com', 'xxxxxx')

# 2. 准备数据
# 创建邮件对象
msg = MIMEMultipart()

# 设置邮件主题
subject = Header('图片发送', 'utf-8').encode()
msg['Subject'] = subject

# 设置邮件发送者
msg['From'] = 'xxxx@163.com <xxxx@163.com>'

# 设置邮件接受者
msg['To'] = '726550822@qq.com'

# 添加图片附件
image_data = open('files/image.jpg', 'rb').read()
image = MIMEImage(image_data)
image["Content-Disposition"] = 'attachment; filename="image.jpg"'	# 设置图片标题
msg.attach(image)

# 3.发送邮件
con.sendmail('xxxx@163.com', '726550822@qq.com', msg.as_string())
con.quit()


以邮件正文形式发送图片

In [ ]:
"""__author__=余婷"""
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.header import Header
from copy import deepcopy

# 1. 连接邮箱服务器
con = smtplib.SMTP_SSL('smtp.163.com', 465)

# 2. 登录邮箱
con.login('xxxx@163.com', 'xxxxxx')

# 2. 准备数据
# 创建邮件对象
msg = MIMEMultipart()

# 设置邮件主题
subject = Header('图片发送', 'utf-8').encode()
msg['Subject'] = subject

# 设置邮件发送者
msg['From'] = 'xxxx@163.com <xxxx@163.com>'

# 设置邮件接受者
msg['To'] = '726550822@qq.com'

# 添加文字内容
content = """
<h2>我是正文中的标题</h2>
<p>邮件正文描述性文字1</p>
<p>邮件正文描述性文字2</p>
<img src='cid:image1'>
<center>图片显示</center>
<a href='https://www.baidu.com'>百度一下</a>
"""
html = MIMEText(content, 'html', 'utf-8')
msg.attach(html)

# 添加图片附件
image_data = open('files/image.jpg', 'rb').read()
image = MIMEImage(image_data)
image.add_header('Content-ID', '<image1>')	
msg.attach(image)

# 3.发送邮件
con.sendmail('xxxx@163.com', '726550822@qq.com', msg.as_string())
con.quit()


发送附件

In [ ]:
"""__author__=余婷"""
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.header import Header
from email.mime.message import MIMEMessage
from copy import deepcopy

# 1. 连接邮箱服务器
con = smtplib.SMTP_SSL('smtp.163.com', 465)

# 2. 登录邮箱
con.login('xxxx@163.com', 'xxxxxx')

# 2. 准备数据
# 创建邮件对象
msg = MIMEMultipart()

# 设置邮件主题
subject = Header('文件附件发送', 'utf-8').encode()
msg['Subject'] = subject

# 设置邮件发送者
msg['From'] = 'xxxx@163.com <xxxx@163.com>'

# 设置邮件接受者
msg['To'] = '726550822@qq.com'

# 添加文件附件
file1 = MIMEText(open('files/test.txt', 'rb').read(), 'base64', 'utf-8')
file1["Content-Disposition"] = 'attachment; filename="test.txt"'
msg.attach(file1)

file2 = MIMEText(open('files/pyecharts的使用.pdf', 'rb').read(), 'base64', 'utf-8')
file2["Content-Disposition"] = 'attachment; filename="test.pdf"'
msg.attach(file2)


# 3.发送邮件
con.sendmail('xxxx@163.com', '726550822@qq.com', msg.as_string())
con.quit()


# 五、使用python直接发邮件的代码

In [6]:
import xlwings as xw
# import win32com.client as win32
import  os
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage


# 完成时间 2021/6/12 端午假期
# 首先定义所有需要用到的文件路径
file_path = '台账模板.xlsx' # 分配表文件所在路径，由于我将文件都拷贝到当前文件夹，所以直接写文件名即可
sheet_name = '模板' # 需要拆分的sheet名称，根据实际情况确定
file_path_investors = r'D:\learnpython\excel_examples\投资者信息统计.xlsx' # 投资者信息所在路径
report = r'D:\learnpython\excel_examples\XX项目受托报告202106.pdf'


#打开excel，读取分配总表数据
app = xw.App(visible = False, add_book = False) # 启动Excel程序
workbook = app.books.open(file_path) # 打开分配表来源工作簿
worksheet = workbook.sheets[sheet_name] # 选中要拆分的数据所在sheet
value = worksheet.range('B38').expand('table').value # 读取需要拆分的所有数据，B38单元格是数据的起始单元格，根据实际情况修改
data = dict() # 创建一个空字典，用于按要求分类储存数据
#打开投资者信息数据
workbook_1 = app.books.open(file_path_investors) # 打开投资者信息工作簿
worksheet_1 = workbook_1.sheets['Sheet1'] # 选中提取的数据所在sheet
value_1 = worksheet_1.range('A1').expand('table').value # 读取需要投资者信息数据
# 拆分分配数据
for i in range(len(value)): # 按行遍历工作表数据
    client_name = value[i][0] # 获取分类依据的数据，两个[]的数字代表行和列，从0开始
    if client_name not in data: #判断字典中是否已有当前客户名称
        data[client_name] = [] # 若没有，就创建一个当前客户名称的空列表，用来储存当前行数据
    data[client_name].append(value[i]) # 将当前行数据追加到当前行客户名称对应的列表中

for key,value in data.items(): # 按客户名称遍历分类后的数据
    new_workbook = xw.books.add() # 新建工作簿
    new_worksheet = new_workbook.sheets.add(key) # 在新工作簿中新建工作表，并且命名为客户名称
    new_worksheet['A1'].value = worksheet['B37:H37'].value # 将要拆分的列标题复制到第一行,根据实际情况修改
    new_worksheet['A2'].value = value # 将当前客户名称下的数据复制到新建的工作表
    new_worksheet.autofit() #自动调整格式 
    new_worksheet.range('B2').number_format = '_ * #,##0.00_ ;_ * -#,##0.00_ ;_ * "-"??_ ;_ @_ ' #设置数字格式
    new_worksheet.range('D2:G2').number_format = '_ * #,##0.00_ ;_ * -#,##0.00_ ;_ * "-"??_ ;_ @_ '
    new_worksheet.range('C2').number_format = '0.00%' 
    # 保存为新工作簿，命名为客户名称，这里也可以加上当前日期或其他内容，根据实际需要调整    
    new_workbook.save(r'D:\learnpython\excel_examples\分配表\{}.xlsx'.format(key)) 
    print('{}已拆分完毕'.format(key))

    # 匹配投资者收件箱，发送邮件
    for j in range(len(value_1)): # 对投资者信息遍历
        if value_1[j][0] == key: # 投资者名称如果和分配表文件名一致
            # 发送邮件
            #设置登录及服务器信息
            mail_host = 'smtp.utrust.cn' #公司邮箱服务器
            mail_user = 'zhaoyunchao@utrust.cn' #发件人邮箱
            mail_pass = 'Yc888888' #自己的密码
            sender = 'zhaoyunchao@utrust.cn'#发件人邮箱
            receivers = [value_1[j][4]] #收件人邮箱，目前用我的邮箱代替，测试用

            #设置eamil信息
            #添加一个MIMEmultipart类，处理正文及附件
            message = MIMEMultipart()
            message['From'] = sender
            message['To'] = receivers[0]
            message['Subject'] = 'XX项目分配邮件通知'
            #推荐使用html格式的正文内容，这样比较灵活，可以附加图片地址，调整格式等
#             with open('abc.html','rb') as f:
#                 content = f.read()
            content = """
#             <p><b>尊敬的投资者：</b></p>
#             <p>您好，请查收贵司本次投资分配明细表，以及受托报告，详见附件。</p>
#             <p>如有任何问题，请及时联系我们。</p>
#             <font color="gray" size="2">赵运超</font><br>
#             <font color="gray" size="2">手机：18924588577/13902319827</font><br/>
#             """
            #设置html格式参数
            part1 = MIMEText(content,'html','utf-8')
            #添加一个txt文本附件
            file1 = r'D:\learnpython\excel_examples\分配表\{}.xlsx'.format(key)
            
            with open(file1,'rb')as h:
                content2 = h.read()
            #设置txt参数
            part2 = MIMEText(content2,'plain','utf-8')
            #附件设置内容类型，方便起见，设置为二进制流
            part2['Content-Type'] = 'application/octet-stream'
            #设置附件头，添加文件名
            part2['Content-Disposition'] = 'attachment; filename = file1.xlsx'
            with open(report,'rb')as g:
                content3 = g.read()
            #设置txt参数
            part3 = MIMEText(content3,'plain','utf-8')
            #附件设置内容类型，方便起见，设置为二进制流
            part3['Content-Type'] = 'application/octet-stream'
            #设置附件头，添加文件名
            part3['Content-Disposition'] = 'attachment; filename = report.pdf'
            #添加照片附件
            # with open('1.png','rb')as fp:
            #     picture = MIMEImage(fp.read())
            #     #与txt文件设置相似
            #     picture['Content-Type'] = 'application/octet-stream'
            #     picture['Content-Disposition'] = 'attachment;filename="1.png"'
            #将内容附加到邮件主体中
            message.attach(part1)
            message.attach(part2)
            message.attach(part3)
            # message.attach(picture)

            #登录并发送
            try:
                smtpObj = smtplib.SMTP()
                smtpObj.connect(mail_host,25)
                smtpObj.login(mail_user,mail_pass)
                smtpObj.sendmail(
                    sender,receivers,message.as_string())
                print('{}邮件已发送'.format(key))
                smtpObj.quit()
            except smtplib.SMTPException as e:
                print('error',e)
#             outlook = win32.Dispatch('Outlook.Application') #初始化outlook
#             mail = outlook.CreateItem(0) # 固定写法
            
#             mail.To = value_1[j][4] # 邮箱收件人名称就是邮箱
#             mail.CC = ''
#             mail.Subject = 'XX项目分配邮件通知'
#             mail.HtmlBody = """
#             <p><b>尊敬的投资者：</b></p>
#             <p>您好，请查收贵司本次投资分配明细表，以及受托报告，详见附件。</p>
#             <p>如有任何问题，请及时联系我们。</p>
#             <font color="gray" size="2">赵运超</font><br>
#             <font color="gray" size="2">手机：18924588577/13902319827</font><br/>
#             """
#             path1 = r'D:\learnpython\excel_examples\分配表\{}.xlsx'.format(key)
#             path2 = report
             
#             mail.Attachments.Add(path1) # 附件1，分配表
#             mail.Attachments.Add(path2) # 附件2，受托报告

#         #     mail.Send()#发送# 应先在邮箱通讯录中建立邮件组，直接输入邮件组名字可以发送
#             mail.Save() #保存草稿
#             print('{}邮件已发送'.format(key))
#             print('找不到投资者{}，请确保分配表中的投资者名称与信息表中完全一致'.format(key))
    
app.quit() # 退出Excel程序

print("*"*27)
print('未提示已发送的，请确保分配表中的投资者名称与信息表中完全一致')
print("已全部拆分完毕,并完成邮件发送")
print("*"*27)

北京银行已拆分完毕
北京银行邮件已发送
兴业银行已拆分完毕
中信银行已拆分完毕
上饶银行已拆分完毕
长沙银行已拆分完毕
广州银行已拆分完毕
台州银行已拆分完毕
小计已拆分完毕
招商证券已拆分完毕
兴业银行长春分行已拆分完毕
东莞银行已拆分完毕
国泰君安证券已拆分完毕
国盛证券已拆分完毕
越秀金控资本管理公司已拆分完毕
中金公司已拆分完毕
兴业资产管理公司已拆分完毕
***************************
未提示已发送的，请确保分配表中的投资者名称与信息表中完全一致
已全部拆分完毕,并完成邮件发送
***************************
